# Data Processing

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Activate below when using Udacity Workspace
# PATH            = '/opt/carnd_p3/data/'
# data            = pd.read_csv("/opt/carnd_p3/data/driving_log.csv")
    
# Activate below when using Local Workspace
PATH            = './data/IMG/'
data            = pd.read_csv("./data/driving_log.csv")

images_center   = []
images_left     = []
images_right    = []

for line in data.loc[:, 'center':'right']:
    for f_center, f_left, f_right in line:
        f_center    = f_center.split('/')[-1]
        f_left      = f_center.split('/')[-1]
        f_right     = f_right.split('/')[-1]
        images_center.append(plt.imread(PATH + f_center))
        images_left.append(plt.imread(PATH + f_left))
        images_right.append(plt.imread(PATH + f_right))

images_center   = np.array(images_center)
images_left     = np.array(images_left)
images_right    = np.array(image_right)

# print(images_center.shape)
# print(images_left.shape)
# print(images_right.shape)

# steering        = data.loc[:, 'steering'].to_numpy()

center
left
right


# Model

In [18]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Conv2D(24, kernel_size = (5, 5), strides= (2, 2), padding= 'valid', activation='relu'))
model.add(Conv2D(36, kernel_size = (5, 5), strides= (2, 2), padding= 'valid', activation='relu'))
model.add(Conv2D(48, kernel_size = (5, 5), strides= (2, 2), padding= 'valid', activation='relu'))
model.add(Conv2D(64, kernel_size = (3, 3), strides= (2, 1), padding= 'valid', activation='relu'))
model.add(Conv2D(64, kernel_size = (3, 3), strides= (2, 1), padding= 'valid', activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = 5) 

model.save('model.h5')


In [3]:
import numpy as np
import csv
import cv2

env = 0

# Activate below when using Local Workspace
PATH_LOCAL            = './data/IMG/'
PATH_CSV_LOCAL        = "./data/driving_log.csv"

# Activate below when using Udacity Workspace
PATH_UDC            = '/opt/carnd_p3/data/IMG/'
PATH_CSV_UDC        = "/opt/carnd_p3/data/driving_log.csv"

# PATH
if env == 0:
    PATH        = PATH_LOCAL
    path_csv    = PATH_CSV_LOCAL
    
elif env == 1:
    PATH        = PATH_UDC
    path_csv    = PATH_CSV_UDC
    
else:
    raise('Invalid Argument')


# Reading Data
with open(path_csv, 'r') as f:
    reader  = csv.reader(f)
    data    = [row for row in reader][1:]
print("Reading CSV file completed")

X           = np.zeros(shape=(len(data), 160, 320, 3), dtype=np.float32)
y_steer     = np.zeros(shape=(len(data),), dtype=np.float32)


for idx, row in enumerate(data):
    ct_path, lt_path, rt_path, steer, throttle, brake, speed = row

    f       = ct_path.split('/')[-1]
    image   = np.float32(cv2.imread(PATH + f))
    steer   = steer

    X[idx] = image
    y_steer[idx] = steer

# from keras.models import Sequential, Model
# from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D, Dropout

# X_train = X
# y_train = y_steer

# model = Sequential()
# model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
# model.add(Conv2D(24, kernel_size = (5, 5), strides= (2, 2), padding= 'valid', activation='relu'))
# model.add(Conv2D(36, kernel_size = (5, 5), strides= (2, 2), padding= 'valid', activation='relu'))
# model.add(Conv2D(48, kernel_size = (5, 5), strides= (2, 2), padding= 'valid', activation='relu'))
# model.add(Conv2D(64, kernel_size = (3, 3), strides= (2, 1), padding= 'valid', activation='relu'))
# model.add(Conv2D(64, kernel_size = (3, 3), strides= (2, 1), padding= 'valid', activation='relu'))
# model.add(Flatten())
# model.add(Dropout(0.2))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation='relu'))
# model.add(Dense(1))

# print(model.summary())

# model.compile(loss = 'mse', optimizer = 'adam')
# model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = 5) 

# model.save('model.h5')

Reading CSV file completed


In [7]:
print(X.shape, y_steer.shape)
print(type(X), y_steer.dtype)

(8036, 160, 320, 3) (8036,)
<class 'numpy.ndarray'> float32
